In [1]:
import tarfile
import os
import pathlib
import numpy as np
from time import time
# temp
import sys
from tqdm.notebook import tqdm
sys.path.append('/workspace/shared_workspace/deep-learning-models/models/vision/detection')

import tensorflow_addons as tfa
import tensorflow as tf
import horovod.tensorflow as hvd

from awsdet.models.detectors.mask_rcnn import MaskRCNN
from awsdet.datasets import DATASETS, build_dataloader
from awsdet.datasets import build_dataset, build_dataloader
from awsdet.models import build_detector
from awsdet.utils.schedulers import schedulers
from awsdet.core import CocoDistEvalmAPHook, CocoDistEvalRecallHook
from awsdet.utils.runner.hooks.logger import tensorboard, text
from awsdet.utils.runner.hooks import checkpoint, iter_timer, visualizer
from awsdet.apis.train import parse_losses, batch_processor, build_optimizer, get_root_logger
from awsdet.utils.misc import Config
import horovod.tensorflow as hvd
from awsdet.utils.runner import sagemaker_runner
from awsdet.utils.schedulers.schedulers import WarmupScheduler
import argparse

from awsdet.utils import visualize
from pycocotools.cocoeval import COCOeval

from awsdet.core.bbox import transforms
import matplotlib.pyplot as plt

from awsdet.utils.fileio import load, dump
import os.path as osp
from awsdet.core.evaluation.coco_utils import fast_eval_recall, results2json
from awsdet.core.mask.transforms import mask2result
from awsdet.core.evaluation import coco_utils
##########################################################################################
# Setup horovod and tensorflow environment
##########################################################################################

fp16 = True
hvd.init()
tf.config.experimental_run_functions_eagerly(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": fp16})
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')


In [2]:
cfg = Config.fromfile("/workspace/shared_workspace/deep-learning-models/models/vision/detection/configs/docker_mask_rcnn.py")

In [7]:
cfg.batch_size_per_device = 2
cfg.workers_per_gpu = 1
cfg.global_batch_size = cfg.batch_size_per_device * hvd.size()
datasets = build_dataset(cfg.data.train)
tf_datasets = [build_dataloader(datasets,
                     cfg.batch_size_per_device,
                     cfg.workers_per_gpu,
                     num_gpus=hvd.size(),
                     dist=True)]

loading annotations into memory...
Done (t=17.36s)
creating index...
index created!


In [8]:
val_dataset = build_dataset(cfg.data.val)
val_tdf, val_size = build_dataloader(val_dataset,
                                     cfg.batch_size_per_device,
                                     cfg.workers_per_gpu,
                                     num_gpus=hvd.size(),
                                     dist=True,
                                     shuffle=False)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


In [9]:
model = build_detector(cfg.model,
                           train_cfg=cfg.train_cfg,
                           test_cfg=cfg.test_cfg)

In [10]:
_ = model(next(iter(tf_datasets[0][0])), training=True)

Starting new loop for GPU: 0


In [11]:
cfg.base_learning_rate = 0.01
cfg.warmup_init_lr_scale = 3.0
cfg.warmup_steps = 500
base_learning_rate = cfg.base_learning_rate
scaled_learning_rate = base_learning_rate * cfg.global_batch_size / 8
steps_per_epoch = 100
scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            [steps_per_epoch * 8, steps_per_epoch * 10],
            [scaled_learning_rate, scaled_learning_rate*0.1, scaled_learning_rate*0.01])
warmup_init_lr = 1.0 / cfg.warmup_init_lr_scale * scaled_learning_rate
scheduler = WarmupScheduler(scheduler, warmup_init_lr, cfg.warmup_steps)
optimizer = tf.keras.optimizers.SGD(scheduler, momentum=0.9, nesterov=False)
optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer, loss_scale='dynamic')

In [12]:
runner = sagemaker_runner.Runner(model, batch_processor, name='mask_test', 
                                     optimizer=optimizer, work_dir='/workspace/shared_workspace/logs',
                                     logger=get_root_logger(cfg.log_level), amp_enabled=True,
                                     loss_weights=cfg.loss_weights, with_mask=True)
runner.register_hook(iter_timer.IterTimerHook())
runner.register_hook(text.TextLoggerHook())
runner.register_hook(CocoDistEvalmAPHook(cfg.data.val, interval=1))

loading annotations into memory...
Done (t=0.40s)
creating index...
index created!


In [14]:
runner.load_checkpoint('/workspace/shared_workspace/output_spike_2/007/mask_rcnn')

2020-06-21 18:22:25,799 - INFO - Loading checkpoint from /workspace/shared_workspace/output_spike_2/007/mask_rcnn...
2020-06-21 18:22:26,840 - INFO - Loaded weights from checkpoint: /workspace/shared_workspace/output_spike_2/007/mask_rcnn


In [21]:
runner.model.layers[1]

In [10]:
#runner.run(tf_datasets, cfg.workflow, 5)

In [11]:
steps = 10
results = []
masks = []
for i, batch in tqdm(enumerate(val_tdf.take(steps)), total=steps):
    img, _ = batch
    detection = runner.model(batch, training=False)
    bboxes = transforms.bbox_mapping_back(detection['bboxes'], batch[1][0])
    box_ints = tf.round(bboxes)
    labels = detection['labels']
    scores = detection['scores']
    result = transforms.bbox2result(bboxes, labels, 
                                          scores, num_classes=val_dataset.CLASSES+1)
    #masks = mask2result(bboxes, detection['masks'], labels, batch[1][0])
    results.append(result)
    masks.append(detection['masks'])

Starting new loop for GPU: 0



In [18]:
import numpy as np
import cv2
from collections import defaultdict
import pycocotools.mask as mask_util

def box_clip(bboxes, img_size):
    y1, x1, y2, x2 = np.split(bboxes, 4, axis=1)
    cy1 = np.clip(y1, 0, img_size[0])
    cx1 = np.clip(x1, 0, img_size[1])
    cy2 = np.clip(y2, 0, img_size[0])
    cx2 = np.clip(x2, 0, img_size[1])
    clipped_boxes = np.transpose(np.squeeze(np.stack([cy1, cx1, cy2, cx2])))
    '''cheight = cy2 - cy1
    cwidth = cx2 - cx1
    crop_height_start = np.abs(y1)
    crop_width_start = np.abs(x1)'''
    return clipped_boxes

def compute_pads(bboxes, img_size):
    y1, x1, y2, x2 = np.split(bboxes, 4, axis=1)
    y1_pad = y1
    x1_pad = x1
    y2_pad = img_size[0] - y2
    x2_pad = img_size[1] - x2
    pads = np.transpose(np.squeeze(np.stack([y1_pad, x1_pad, y2_pad, x2_pad])))
    return pads

def fit_pad(mask, pad):
    mask = np.pad(mask, ((pad[0], pad[2]), (pad[1], pad[3])))
    return mask

def fit_mask_to_image(mask, cv2_size, clipped_box_size, 
                      clipped_boxes_np, pad, img_shape):
    if np.multiply(*clipped_box_size)==0:
        return np.zeros(img_shape, dtype=np.int32)
    mask = cv2.resize(mask, tuple(cv2_size), interpolation=cv2.INTER_NEAREST)
    if mask.ndim==1:
        mask = np.expand_dims(mask, axis=np.argmin(clipped_box_size))
    mask = mask[:clipped_box_size[0],:clipped_box_size[1]]
    #print('\n')
    #print(mask.shape)
    mask = fit_pad(mask, pad)
    #print(mask.shape)
    #print(clipped_boxes_np)
    return mask

def reshape_by_labels(mask_list, labels, num_classes=81):
    list_of_lists = [[]]*num_classes
    for mask, label in zip(mask_list, labels):
        list_of_lists[label].append(mask)
    return list_of_lists

def mask2result(bboxes, masks, labels, meta, num_classes=81, threshold=0.5):
    # convert tensors to numpy
    # round bboxes to nearest int
    meta = np.squeeze(meta)
    img_heights, img_widths = meta[:2].astype(np.int32)
    bboxes_np = np.round(bboxes).astype(np.int32)
    clipped_boxes_np = box_clip(bboxes_np, (img_heights, img_widths))
    masks_np = (masks.numpy()>threshold).astype(np.int32)
    labels_np = labels.numpy()
    if meta[-1]==1:
        masks_np = np.flip(masks_np, axis=2)
    bbox_heights = bboxes_np[:,2]-bboxes_np[:,0]
    bbox_clipped_heights = clipped_boxes_np[:,2]-clipped_boxes_np[:,0]
    bbox_widths = bboxes_np[:,3]-bboxes_np[:,1]
    bbox_clipped_widths = clipped_boxes_np[:,3]-clipped_boxes_np[:,1]
    bbox_sizes = np.transpose(np.stack([bbox_heights, bbox_widths]))
    #cv2 needs dims in opposite direction
    cv2_sizes = np.flip(bbox_sizes, axis=1)
    bbox_clipped_sizes = np.squeeze(np.transpose([np.stack([bbox_clipped_heights,
                                                 bbox_clipped_widths])]))
    pads = compute_pads(clipped_boxes_np, (img_heights, img_widths))
    
    mask_list = []
    for idx in range(100):
        #print(idx)
        mask_list.append(fit_mask_to_image(masks_np[idx],
                                           cv2_sizes[idx],
                                           bbox_clipped_sizes[idx],
                                           clipped_boxes_np[idx],
                                           pads[idx],
                                           (img_heights, img_widths)))
    lists = defaultdict(list)
    for i,j in enumerate(labels_np):
        lists[j].append(mask_util.encode(
                    np.array(
                        mask_list[i][:, :, np.newaxis], order='F',
                        dtype='uint8'))[0])
    return lists

In [20]:
%%timeit
mask_list = \
mask2result(bboxes, detection['masks'], labels, batch[1])

82.8 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
steps = 500
masks_list = []
result_list = []
for i, batch in tqdm(enumerate(val_tdf.take(steps)), total=steps):
    detection = runner.model(batch, training=False)
    bboxes = transforms.bbox_mapping_back(detection['bboxes'], batch[1][0])
    box_ints = tf.round(bboxes)
    labels = detection['labels']
    scores = detection['scores']
    result = transforms.bbox2result(bboxes, labels, 
                                          scores, num_classes=val_dataset.CLASSES+1)
    masks = mask2result(box_ints, detection['masks'], labels, batch[1][0])
    #masks_list.append(masks)
    result_list.append((result, masks))

Starting new loop for GPU: 0


In [ ]:
result_list

In [42]:
from importlib import reload
coco_utils = reload(coco_utils)

In [43]:
result_files = coco_utils.results2json(val_dataset, results, 'temp_eval')

In [44]:
result_files

{'bbox': 'temp_eval.bbox.json', 'segm': 'temp_eval.segm.json'}

In [16]:
for detection in detections:
    bboxes = tf.round(transforms.bbox_mapping_back(detections['bboxes'], batch[1][0]))
    #bboxes_round = tf.round(transforms.bbox_mapping_back(detections['bboxes'], batch[1][0]))

TypeError: list indices must be integers or slices, not str

In [46]:
runner.model.